In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('bmi_data.csv')

In [3]:
data.head()

In [4]:
data.isnull().sum()

In [5]:
data = data[(data != 0).all(axis=1)]

In [6]:
condition = (data[['Height(Inches)','Weight(Pounds)']] == 0).any(axis=1)

# Menghapus baris yang memenuhi kondisi
data = data.drop(data[condition].index)
print("\nDataFrame setelah penghapusan:")
print(data)

In [7]:
data['Sex'] = data['Sex'].map({'Male': 1, 'Female': 0})

In [8]:
data.head()

In [9]:
#mengubah data height dan weight untukbmr
data['Height(CM)'] = data['Height(Inches)'] * 2.54
data['Weight(KG)'] = data['Weight(Pounds)'] * 0.453592

In [10]:
data.head()

In [11]:
columns_to_drop = ['BMI', 'Height(Inches)', 'Weight(Pounds)']
data = data.drop(columns=columns_to_drop)

In [12]:
data.head()

In [13]:
columns_to_check = ['Height(CM)', 'Weight(KG)']
data = data.dropna(subset=columns_to_check, how='any')

In [14]:
data.info()

In [15]:
# fungsi untuk menghitung bmr
def calculate_bmr(row):
    if row['Sex'] == 1:
        return 66 + (13.7 * row['Weight(KG)']) + (5 * row['Height(CM)']) - (6.8 * row['Age'])
    elif row['Sex'] == 0:
        return 665 + (9.6 * row['Weight(KG)']) + (1.8 * row['Height(CM)']) - (4.7 * row['Age'])
    else:
        return None 

In [16]:
data['BMR'] = data.apply(calculate_bmr, axis=1)

In [17]:
data.head()

In [18]:
data.describe()

In [19]:
data.info()

In [20]:
# Mengatur ulang batas bin untuk 3 kategori
bins = [data['BMR'].min(), data['BMR'].quantile(0.33), data['BMR'].quantile(0.66), data['BMR'].max()]
labels = ['Rendah', 'Sedang', 'Tinggi']

# Menambahkan kolom baru dengan kategori berdasarkan nilai BMR
data['BMR_Category'] = pd.cut(data['BMR'], bins=bins, labels=labels)


In [21]:
from sklearn.preprocessing import LabelEncoder

# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Melakukan encoding pada kolom BMR_Category
data['BMR_Category_Encoded'] = label_encoder.fit_transform(data['BMR_Category'])

In [22]:
data.info()

In [23]:
data.head()

In [24]:
data.BMR_Category_Encoded.unique()

In [25]:
data = data.drop(index=22945)

In [26]:
data.BMR_Category_Encoded.unique()

### membuat model untuk data


In [27]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [28]:
X = data.drop(columns=['BMR','BMR_Category_Encoded','BMR_Category'])

In [29]:
Y = data['BMR_Category_Encoded']
print(Y)

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [31]:
from sklearn.preprocessing import StandardScaler
# Normalisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_dim=X_train.shape[1], activation='relu'),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(5,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Melatih model
model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_data=(X_test, Y_test))

_, accuracy = model.evaluate(X_test, Y_test)
print(f'Akurasi model: {accuracy}')

In [33]:
# Data baru untuk prediksi (ganti nilainya sesuai dengan data yang ingin diprediksi)
new_data = {
    'Sex': 1,
    'Age': 30,
    'Height(CM)': 170,
    'Weight(KG)': 60,
    # Jika kolom 'BMR' tidak ada, ganti nilainya sesuai dengan data aktual
}

# Mengubah data input menjadi bentuk yang dapat diproses oleh model
new_data_df = pd.DataFrame([new_data])  # Ubah data menjadi DataFrame jika belum
new_data_processed = scaler.transform(new_data_df)  # Normalisasi fitur seperti yang dilakukan pada data latih

# Melakukan prediksi dengan model yang sudah dilatih
prediction = model.predict(new_data_processed)
print(f'Prediksi kategori BMR: {prediction}')


In [34]:
predicted_category = np.argmax(prediction)
print(f'Kategori BMR diprediksi: {predicted_category}')

In [35]:
data.BMR_Category_Encoded.unique()

In [36]:
data.to_csv('bmi_clean',index=False)

In [37]:
data.info()

In [38]:
data.describe()